In [45]:
import numpy as numpy
import pandas as pd
import os
import numpy as np
!pip install rapidfuzz
import re
import datetime as dt
import glob
from math import radians, sin, cos, sqrt, atan2

In [46]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers

    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)

    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [47]:
delays = pd.read_csv('bus_delay_raw.csv')
fuzz = pd.read_csv('fuzz_locations.csv')
stops = pd.read_csv('stops.txt')
intersections = pd.read_csv('inter_coord.csv')

C:\Users\gaspa\AppData\Local\Temp\ipykernel_5748\3509189521.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  delays = pd.read_csv('bus_delay_raw.csv')


In [48]:
from rapidfuzz import process
def find_closest_match_threshold(row, reference_list, threshold = 40):
    match = process.extractOne(row, reference_list)
    if match[1] < threshold:
        return 'not able to fuzz'
    else:
        return match[0]

In [51]:
intersections.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33695 entries, 0 to 33694
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           33695 non-null  int64  
 1   INTERSECTION_DESC    33695 non-null  object 
 2   CLASSIFICATION_DESC  33695 non-null  object 
 3   geometry             33695 non-null  object 
 4   lat                  33695 non-null  float64
 5   lon                  33695 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.5+ MB


In [53]:
fuzz['fuzzed'] = fuzz['fuzz_locations'].apply(find_closest_match_threshold, args=(intersections['INTERSECTION_DESC'],))
fuzz.to_csv('temp_fuzz.csv')
fuzz = pd.merge(fuzz, intersections, left_on='fuzzed', right_on='INTERSECTION_DESC', how='left')

for index, row in fuzz.iterrows():
    lat_location, lon_location = row['lat'], row['lon']
    
    # Calculate distances to each stop and find the closest one
    stops['distance'] = stops.apply(lambda stop: haversine(lat_location, lon_location, stop['stop_lat'], stop['stop_lon']), axis=1)
    
    # Find the closest stop row
    closest_stop = stops.loc[stops['distance'].idxmin()]
    
    # Assign the closest stop information to the corresponding columns in 'fuzz'
    fuzz.at[index, 'closest_stop'] = closest_stop['stop_name']
    fuzz.at[index, 'stop_lon'] = closest_stop['stop_lon']
    fuzz.at[index, 'stop_lat'] = closest_stop['stop_lat']
    
fuzz.to_csv('fuzz_geo.csv')

In [54]:
fuzz.head()

,Unnamed: 0_x,fuzz_locations,fuzzed,Unnamed: 0_y,INTERSECTION_DESC,CLASSIFICATION_DESC,geometry,lat,lon,closest_stop,stop_lon,stop_lat
0,0,lawrence&warden,lawrenceavee&wardenave,10975,lawrenceavee&wardenave,Major-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.29...",43.745377,-79.294730,Lawrence Ave East at Warden Ave,-79.294565,43.745551
1,1,ellesmere&victoriapark,ellesmererd,8450,ellesmererd,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.27...",43.766165,-79.279371,Ellesmere Rd at West Service Rd East Side,-79.278977,43.766111
2,2,markham&passmore,markhamrd&passmoreave,32497,markhamrd&passmoreave,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.25...",43.830549,-79.250717,Markham Rd at Passmore Ave,-79.250916,43.830781
3,3,lawrence&scarboroughgolfclubrd,lawrenceavee&scarboroughgolfclubrd,32779,lawrenceavee&scarboroughgolfclubrd,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.21...",43.761852,-79.215838,Scarborough Golf Club Rd at Lawrence Ave East,-79.215999,43.762027
4,4,danforthave&victoriapark,danforthave&victoriaparkave,13993,danforthave&victoriaparkave,Major-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.28...",43.691232,-79.288359,Victoria Park Ave at Danforth Ave,-79.288525,43.691357


In [55]:
fuzz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34859 entries, 0 to 34858
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0_x         34859 non-null  int64  
 1   fuzz_locations       34859 non-null  object 
 2   fuzzed               34859 non-null  object 
 3   Unnamed: 0_y         34859 non-null  int64  
 4   INTERSECTION_DESC    34859 non-null  object 
 5   CLASSIFICATION_DESC  34859 non-null  object 
 6   geometry             34859 non-null  object 
 7   lat                  34859 non-null  float64
 8   lon                  34859 non-null  float64
 9   closest_stop         34859 non-null  object 
 10  stop_lon             34859 non-null  float64
 11  stop_lat             34859 non-null  float64
dtypes: float64(4), int64(2), object(6)
memory usage: 3.2+ MB


In [58]:
geo_delays = pd.merge(delays,fuzz, left_on='copy', right_on='fuzz_locations', how='left')

# Create the 'analysis' column and fill it with 'fuzz' if 'location' is not NaN, otherwise 'not fuzzed'
geo_delays['analysis'] = geo_delays['fuzzed'].where(~geo_delays['fuzz_locations'].isna(), geo_delays['copy'])

#columns_to_drop = ['fuzz_locations', 'fuzzed','copy']
#bus_delays = bus_delays.drop(columns=columns_to_drop)

In [59]:
geo_delays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588589 entries, 0 to 588588
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           588589 non-null  int64  
 1   Route                588511 non-null  object 
 2   Day                  588589 non-null  object 
 3   Location             587796 non-null  object 
 4   Incident             587654 non-null  object 
 5   Min Delay            551142 non-null  float64
 6   Min Gap              556190 non-null  float64
 7   Vehicle              518753 non-null  float64
 8   DateTime             577298 non-null  object 
 9   copy                 587769 non-null  object 
 10  Unnamed: 0_x         198030 non-null  float64
 11  fuzz_locations       198030 non-null  object 
 12  fuzzed               198030 non-null  object 
 13  Unnamed: 0_y         198030 non-null  float64
 14  INTERSECTION_DESC    198030 non-null  object 
 15  CLASSIFICATION_DE